# **Estacionalidad**

In [8]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "vallarta" in nombre and '2019' not in nombre and '2020' not in nombre and '2021' not in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2022_vallarta',
 'df_financiamientos_2023_vallarta',
 'df_financiamientos_2024_vallarta']

In [9]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id', 'año', y 'mes'
    segment_df = globals()[nombre_df][['id','año','mes']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)
# Concatenar todos los DataFrames en la lista en un solo DataFrame
estacionalidad_2223 = pd.concat(dataframes_list, ignore_index=True)
estacionalidad_2223

,id,año,mes
0,1,2022,1
1,2,2022,1
2,3,2022,1
3,4,2022,1
4,5,2022,1
...,...,...,...
3833,1176,2024,11
3834,1177,2024,11
3835,1178,2024,11
3836,1179,2024,11


In [10]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
estacionalidad_2223['trimestre'] = estacionalidad_2223['mes'].apply(asignar_trimestre)
estacionalidad_2223

,id,año,mes,trimestre
0,1,2022,1,1
1,2,2022,1,1
2,3,2022,1,1
3,4,2022,1,1
4,5,2022,1,1
...,...,...,...,...
3833,1176,2024,11,4
3834,1177,2024,11,4
3835,1178,2024,11,4
3836,1179,2024,11,4


In [11]:
conteo_trimestral = estacionalidad_2223.groupby(['año', 'trimestre']).size().reset_index(name='conteo')
conteo_trimestral = conteo_trimestral[conteo_trimestral['año'] != 0]
# Concatenar 'año' y 'trimestre' en una nueva columna llamada 'trimestre'
conteo_trimestral['trimestre'] = conteo_trimestral['trimestre'].astype(str) + 'o ' + conteo_trimestral['año'].astype(str) 
# Eliminar la columna 'año' si ya no es necesaria
conteo_trimestral.drop(columns=['año'], inplace=True)
conteo_trimestral

,trimestre,conteo
0,1o 2022,338
1,2o 2022,269
2,3o 2022,308
3,4o 2022,359
4,1o 2023,362
5,2o 2023,382
6,3o 2023,318
7,4o 2023,322
8,1o 2024,318
9,2o 2024,340


## *OCULTAR 3RO*

In [5]:
#conteo_trimestral = conteo_trimestral[conteo_trimestral['trimestre'] != '3o 2024']


In [14]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff','#ff0059']
import numpy as np
# Calcula el máximo valor de la serie y redondea hacia arriba al múltiplo más cercano de 50
#max_y = np.ceil(conteo_trimestral['conteo'].max() / 50) * 50
#max_y = conteo_trimestral['conteo']

# Crea un rango de valores de y en incrementos de 50 hasta el máximo
#values_range = np.arange(0, max_y + 1, 50)
# Modifica los valores de 'values' en tu gráfico para que estén en incrementos de 50
#values = conteo_trimestral['conteo'].apply(lambda x: min(values_range, key=lambda y: abs(y - x)))
values = conteo_trimestral['conteo']
# Luego, puedes proceder con la creación de tu gráfico como lo hiciste antes
labels = conteo_trimestral['trimestre']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,
        textposition='inside',
        showlegend=False,
        #width=0.5 
    )
)    
fig.update_layout(
    #title='Número de créditos por trimestre (2022-2023)',
    xaxis_title="Trimestre",        
    yaxis=dict(
        title="Créditos",
        gridcolor='#dddcda',
        #tickvals=values_range,  # Especifica los valores de los ticks del eje y
        #tickmode='array'  # Usa los valores especificados en 'tickvals' para los ticks
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)
guardar_grafico_como_html(fig, 'g_bar_estacionalidad', carpeta='assets\graficas')
fig.show()

<>:42: SyntaxWarning:

invalid escape sequence '\g'

<>:59: SyntaxWarning:

invalid escape sequence '\g'

<>:42: SyntaxWarning:

invalid escape sequence '\g'

<>:59: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1716\4023201357.py:42: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1716\4023201357.py:59: SyntaxWarning:

invalid escape sequence '\g'

